In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

import thesis_helper
functions = thesis_helper.Thesis_Helper()

from flair.embeddings import TransformerWordEmbeddings
from flair.data import Sentence

# init embedding
embedding = TransformerWordEmbeddings('Ivo/emscad-skill-extraction-conference-token-classification')

#bert-large-uncased


Some weights of BertModel were not initialized from the model checkpoint at Ivo/emscad-skill-extraction-conference-token-classification and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
df = pd.read_csv('/Users/ivowings/Sync/Thesis/Datasources/Conference data/conference_data.csv',sep=',')
df = df.fillna('')
df['concatenated'] = df['left_context'] + ' | ' + df['recovered_gram'] + ' | ' + df['right_context']
df.head()

,left_context,recovered_gram,right_context,label,concatenated,concatenated2
0,be look for a temporary opportunity within a p...,dynamic,environment please forward your cv today,0,be look for a temporary opportunity within a p...,be look for a temporary opportunity within a p...
1,for over 30 year this client be a lead,independent,supplier of bespeak product and solution to t...,0,for over 30 year this client be a lead | inde...,for over 30 year this client be a lead | inde...
2,you will be responsible for lead the clinical ...,team working,be,0,you will be responsible for lead the clinical ...,you will be responsible for lead the clinical ...
3,rfi rfp bid,proposal writing,empty,0,rfi rfp bid | proposal writing | empty,rfi rfp bid | proposal writing | empty
4,travel managementstrong analytical and organis...,dynamic,and proactive in your sale approach with a re...,0,travel managementstrong analytical and organis...,travel managementstrong analytical and organis...


In [ ]:
def bert_embedder(text,label):

    label = label
    string = Sentence(text)
    embedding.embed(string)


    # Creating a list which stores the indexes of the | symbols
    bar_indexes_skill = []
    #Creating a list which stores the embeddings_tensors_skill
    embeddings_tensors_skill = []
    #Creating a list to store the labels
    labels = []

    #Checking the sentence object for the | symbols and storing their indexes
    for x in range(1,len(string)+1):
        if '|' in str(string.get_token(x)):
            bar_indexes_skill.append(x)

    token_list = []

    #Collecting the embeddings for every index between the indexes in bar_indexes_skill
    word_embedding_indexes = range(bar_indexes_skill[0],bar_indexes_skill[1]-1)
    for x in word_embedding_indexes:
        embeddings_tensors_skill.append(pd.Series(string[x].embedding))
        token_list.append(string[x].text)

    #Turning the elements from embeddings_tensors_skill into dataframe rows
    row_skill = pd.DataFrame()
    for x in range(0,len(embeddings_tensors_skill)):
        indi_row = pd.DataFrame(pd.Series(embeddings_tensors_skill[x]))
        indi_row = indi_row.transpose()
        row_skill = pd.concat([row_skill,indi_row])

    #Changing the column names in order to make pd.concat work later
    row_skill.columns = [x for x in range(0,len(row_skill.columns))]
    row_skill['label'] = label
    row_skill['token'] = token_list

    #This section converts all non labeled tokens into seperate bert embeddings

    # Creating a list which stores the indexes of the | symbols
    bar_indexes = []
    #Creating a list which stores the embedding_tensors
    embedding_tensors2 = []
    #Creating a list to store the labels
    labels = []

    #Checking the sentence object for the | symbols and storing their indexes
    for x in range(1,len(string)+1):
        if '|' in str(string.get_token(x)):
            bar_indexes.append(x)

    #Collecting the embeddings for every index between the indexes in bar_indexes
    word_embedding_indexes = range(bar_indexes[0]-1,bar_indexes[1])
    banaan = range(0,len(string))

    other_words = [x for x in banaan if x not in word_embedding_indexes]
    #other_words = [0,1]

    token_list = []

    for x in other_words:
        embedding_tensors2.append(pd.Series(string[x].embedding))
        token_list.append(string[x].text)


    #Turning the elements from embedding_tensors into dataframe rows
    row = pd.DataFrame()
    for x in range(0,len(embedding_tensors2)):
        indi_row = pd.DataFrame(pd.Series(embedding_tensors2[x]))
        indi_row = indi_row.transpose()
        row = pd.concat([row,indi_row])

    row['label'] = 0
    row['token'] = token_list

    final_row = pd.concat([row_skill,row]).reset_index(drop=True)
    return final_row



In [ ]:
bert_embeddings = pd.DataFrame()

for text,label in tqdm(zip(df['concatenated'],df['label'])):
    bert_embeddings = pd.concat([bert_embeddings,bert_embedder(text,label)])

    

In [8]:
from sklearn.utils import shuffle


bert_embeddings = shuffle(bert_embeddings)

In [20]:
x_bert = bert_embeddings.drop(columns=['token','label'])

In [22]:
%%time

functions.model_performance(x_bert, bert_embeddings['label'])

  0%|          | 0/6 [00:00<?, ?it/s]

Starting model evaluation
We are at classifier  LogisticRegression(max_iter=10000000000000000000000, random_state=456)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 55.5min finished
 17%|█▋        | 1/6 [55:31<4:37:39, 3331.87s/it]

We are at classifier  GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=456)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 412.9min finished
 33%|███▎      | 2/6 [7:48:24<17:42:57, 15944.30s/it]

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 111.6min finished
 83%|████████▎ | 5/6 [13:42:45<2:37:18, 9438.13s/it] 

We are at classifier  MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15,),
              max_iter=10000000000000000000000, random_state=456,
              solver='lbfgs')


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  9.5min finished
100%|██████████| 6/6 [13:52:18<00:00, 8323.00s/it]  


CPU times: user 6.99 s, sys: 24.2 s, total: 31.2 s
Wall time: 13h 52min 18s


,Classifier,Precision,Recall,F1
0,LR,0.880810,0.864470,0.872208
1,GBC,0.334281,0.421515,0.344317
2,SGD,0.909848,0.843280,0.872815
3,RF,0.923061,0.633686,0.693682
4,SVM,0.920099,0.842557,0.876206
5,MLP,0.871169,0.860620,0.865103


In [23]:
x_bert = bert_embeddings.reset_index(drop=True)

x_bert['pos'] = x_bert['token'].progress_apply(functions.pos_tagger)
x_bert['pos'] = x_bert['pos'].progress_apply(functions.sequence_counter)

pos_dicts = x_bert[['pos']]
pos_dicts = pos_dicts['pos'].apply(pd.Series)
pos_dicts = pos_dicts.fillna(0).astype(int)

x_bert['dep'] = x_bert['token'].progress_apply(functions.dep_tagger)
x_bert['dep'] = x_bert['dep'].progress_apply(functions.sequence_counter)

dep_dicts = x_bert[['dep']]
dep_dicts = dep_dicts['dep'].apply(pd.Series)
dep_dicts = dep_dicts.fillna(0).astype(int)

x_pos = pos_dicts.join(dep_dicts,lsuffix='_gram', rsuffix='_pos')

x = x_bert.drop(columns=['token','label','pos','dep']).join(x_pos, lsuffix='_embedding', rsuffix='_pos')

100%|██████████| 205245/205245 [4:42:39<00:00, 12.10it/s]    


In [24]:
%%time
functions.model_performance(x.fillna(0.0), bert_embeddings['label'])

  0%|          | 0/6 [00:00<?, ?it/s]

Starting model evaluation
We are at classifier  LogisticRegression(max_iter=10000000000000000000000, random_state=456)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 170.3min finished
 17%|█▋        | 1/6 [2:50:16<14:11:24, 10216.99s/it]

We are at classifier  GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=456)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 470.5min finished
 33%|███▎      | 2/6 [10:40:49<23:07:37, 20814.47s/it]

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 116.6min finished
 83%|████████▎ | 5/6 [16:49:58<2:58:32, 10712.01s/it]

We are at classifier  MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15,),
              max_iter=10000000000000000000000, random_state=456,
              solver='lbfgs')


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  9.8min finished
100%|██████████| 6/6 [16:59:47<00:00, 10197.98s/it]  


CPU times: user 7.5 s, sys: 23.7 s, total: 31.2 s
Wall time: 16h 59min 49s


,Classifier,Precision,Recall,F1
0,LR,0.879385,0.861406,0.869882
1,GBC,0.334281,0.421515,0.344317
2,SGD,0.909934,0.844978,0.873543
3,RF,0.934330,0.631489,0.691054
4,SVM,0.921364,0.842080,0.876416
5,MLP,0.872919,0.865523,0.868368
